<a href="https://colab.research.google.com/github/Mayakshanesht/Semantic-Segmentation/blob/main/Semantic_segmentation_version3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#lets mount drive on the google colab
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/segmentation/segmentation_course-master/dataset/dataset")
!ls

Mounted at /content/drive
costa_rica_challenge.mp4  labels_3000_160.p  SegmentationTest2.h5
images_3000_160.p	  model.png	     SegmentationTest.h5


In [2]:
#lets import important python libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import cv2
import random
import time
#lets install tensorflow
import tensorflow as tf
print(tf.__version__)

2.6.0


In [3]:
with open('images_3000_160.p','rb') as f:
  images=pickle.load(f)
with open('labels_3000_160.p','rb') as f:
  labels=pickle.load(f) 

In [4]:
masks=[]
for label in labels:
  for x in range(label.shape[0]):
    for y in range(label.shape[1]):
      if(np.all(label[x][y]==[0,0,0])):
        label[x][y]=[0,1,0]
        masks.append(label)

In [7]:
images=(np.array(images[0:2999]))
#images1=np.array(images[0:2999])
masks=(np.array(masks[0:2999]))

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from scipy import io
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt 

In [9]:
base = keras.applications.DenseNet121(input_shape=[128,128,3], 
                                       include_top=False, 
                                       weights='imagenet')
skip_names = ['conv1/relu', # size 64*64
              'pool2_relu',  # size 32*32
              'pool3_relu',  # size 16*16
              'pool4_relu',  # size 8*8
              'relu'        # size 4*4
              ] 
skip_outputs = [base.get_layer(name).output for name in skip_names]
for i in range(len(skip_outputs)):
     print(skip_outputs[i]) 
downstack = keras.Model(inputs=base.input,
                        outputs=skip_outputs)
downstack.trainable = False 
!pip install -q git+https://github.com/tensorflow/examples.git
from tensorflow_examples.models.pix2pix import pix2pix
 # Four upstack blocks for upsampling sizes 
 # 4->8, 8->16, 16->32, 32->64 
upstack = [pix2pix.upsample(512,3),
           pix2pix.upsample(256,3),
           pix2pix.upsample(128,3),
           pix2pix.upsample(64,3)] 
# define the input layer
inputs = keras.layers.Input(shape=[128,128,3])
 # downsample 
down = downstack(inputs)
out = down[-1]
 # prepare skip-connections
skips = reversed(down[:-1])
 # choose the last layer at first 4 --> 8
 # upsample with skip-connections
for up, skip in zip(upstack,skips):
     out = up(out)
     out = keras.layers.Concatenate()([out,skip])
 # define the final transpose conv layer
 # image 128 by 128 with 59 classes
out = keras.layers.Conv2DTranspose(3, 3,
                                   strides=2,
                                   padding='same',
                                   )(out)
 # complete unet model
unet = keras.Model(inputs=inputs, outputs=out)

29097984/29084464 [==============================] - 0s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 64, 64, 64), dtype=tf.float32, name=None), name='conv1/relu/Relu:0', description="created by layer 'conv1/relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 256), dtype=tf.float32, name=None), name='pool2_relu/Relu:0', description="created by layer 'pool2_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 512), dtype=tf.float32, name=None), name='pool3_relu/Relu:0', description="created by layer 'pool3_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 8, 8, 1024), dtype=tf.float32, name=None), name='pool4_relu/Relu:0', description="created by layer 'pool4_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 1024), dtype=tf.float32, name=None), name='relu/Relu:0', description="created by layer 'relu'")
    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install

In [10]:
def resize_image(image):
     image = tf.cast(image, tf.float32)
     # scale values to [0,1]
     image = image/255.0
     # resize image
     image = tf.image.resize(image, (128,128))
     return image 
def resize_mask(mask):
     mask = tf.expand_dims(mask, axis=-1)
     mask = tf.image.resize(mask, (128,128))
     mask = tf.cast(mask, tf.uint8)
     return mask 
X = np.array([resize_image(i) for i in images])
y = np.array([resize_mask(m) for m in masks])

In [12]:
from sklearn.model_selection import train_test_split
train_X, val_X,train_y, val_y = train_test_split(X,y, 
                                                       test_size=0.2, 
                                                       random_state=0
                                                      )

In [13]:
unet.compile(loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
             metrics=['accuracy']) 

In [14]:
unet.fit(train_X,train_y,epochs=1,validation_data=(val_X,val_y))

ValueError: ignored